# <span style="color:green"><center>Aprendizaje Profundo</center></span>

# <span style="color:red"><center>Pytorch</center></span>

<center>Optimización con torch</center>

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Diseño gráfico y Marketing digital</span>
 

1. Maria del Pilar Montenegro Reyes, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span> 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Pipeline de HuggingFace](#Pipeline-de-HuggingFace)

## <span style="color:blue">Introducción</span>

Ahora que tenemos un modelo y datos, es hora de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. 

Entrenar un modelo es un proceso iterativo; en cada iteración (llamada época) el modelo hace una suposición sobre la salida, calcula el error en su suposición (pérdida), recopila las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior) y optimiza estos parámetros usando el descenso de gradiente.

## <span style="color:blue">Implementación completa</span>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transform import ToTensor, Lambda

# datos
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# modelo
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.reLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


# hiperparámetros
learning_rate = 1e-3
batch_size = 64
epochs = 5

# loop de entrenamiento

# función de pérdida
loss_fn = nn.CrossEntropyLoss()

# Optimizador
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# función de entrenamiento
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, X, y in enumerate(dataloader):
        # calcula predicción y pérdida
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropagation
        optimizer.zero_grad()# por defecto el calculodel gradiente es acumulativo
        loss.backward() # clacula y acumula los gradientes
        optimizer.step()# paso de optimización x += grad
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

# función de validación
def test_loop(testloader, model, loss_fn):
    size = len(testloader.dataset)
    num_batches = len(testloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# ciclo de entrenamiento
epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n -------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')
